In [1]:
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
api_key_pw = os.getenv('api_key_bot')
api_secret_pw = os.getenv('api_secret_bot')


In [4]:
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com', 
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
ws = inverse_perpetual.WebSocket(
    test=False,
    api_key= api_key_pw,
    api_secret= api_secret_pw
)

In [5]:
pd.DataFrame(session.latest_information_for_symbol(symbol ='BTCUSD')["result"]).keys()

Index(['symbol', 'bid_price', 'ask_price', 'last_price', 'last_tick_direction',
       'prev_price_24h', 'price_24h_pcnt', 'high_price_24h', 'low_price_24h',
       'prev_price_1h', 'mark_price', 'index_price', 'open_interest',
       'countdown_hour', 'turnover_24h', 'volume_24h', 'funding_rate',
       'predicted_funding_rate', 'next_funding_time',
       'predicted_delivery_price', 'total_turnover', 'total_volume',
       'delivery_fee_rate', 'delivery_time', 'price_1h_pcnt', 'open_value'],
      dtype='object')

In [6]:
session.orderbook(symbol='BTCUSD')['result'][1]

{'symbol': 'BTCUSD', 'price': '17006.5', 'side': 'Buy', 'size': 41}

In [7]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1669847760000,17093.08,17099.28,17081,17082.43,3.737464,1669847820000,63874.64969365,82,0,0
1,1669847820000,17082.43,17082.43,17069,17071.51,2.799931,1669847880000,47804.18146607,65,0,0
2,1669847880000,17071.51,17081,17068.67,17078.23,5.196525,1669847940000,88711.00970503,76,0,0
3,1669847940000,17078.23,17084.84,17078.23,17080.03,1.081537,1669848000000,18473.11798831,45,0,0
4,1669848000000,17080.03,17091.1,17075.89,17091.1,9.78898,1669848060000,167233.60107975,164,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1669907460000,17071.42,17075.79,17048,17061.33,11.331992,1669907520000,193333.66284452,163,0,0
996,1669907520000,17061.33,17061.85,17021.11,17037.2,13.797165,1669907580000,235106.39004928,210,0,0
997,1669907580000,17037.2,17037.77,17010,17010.45,11.558164,1669907640000,196685.9649464,120,0,0
998,1669907640000,17010.45,17022.03,17000.43,17016.76,20.862631,1669907700000,354884.18411211,176,0,0


In [8]:
def get15minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="15m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [9]:
df = get15minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-11-21 05:30:00,16190.00,16231.95,16166.14,16226.37,25.384596
2022-11-21 05:45:00,16226.37,16236.81,16206.89,16227.44,18.979308
2022-11-21 06:00:00,16227.44,16257.90,16207.58,16212.08,31.419193
2022-11-21 06:15:00,16212.08,16219.99,16160.84,16164.00,32.934481
2022-11-21 06:30:00,16164.00,16182.68,16136.36,16149.45,45.519409
...,...,...,...,...,...
2022-12-01 14:15:00,17084.19,17084.19,17025.80,17031.79,123.810470
2022-12-01 14:30:00,17031.79,17087.12,17026.98,17070.34,90.611994
2022-12-01 14:45:00,17070.34,17142.84,17070.34,17107.77,68.125404


In [10]:
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)


In [11]:
apply_technicals(df)

In [12]:
df.head()

,Open,High,Low,Close,Volume,K,RSI
Time,,,,,,,
2022-11-21 12:00:00,16080.07,16086.75,16021.54,16043.33,56.385221,0.605827,41.854381
2022-11-21 12:15:00,16043.33,16060.73,16029.01,16043.68,19.690105,0.608055,41.917252
2022-11-21 12:30:00,16043.68,16114.03,16025.00,16103.06,20.510247,0.947601,51.498894
2022-11-21 12:45:00,16103.06,16136.83,16087.93,16109.98,17.029577,0.969006,52.482669
2022-11-21 13:00:00,16109.98,16202.06,16096.81,16178.39,54.096590,1.000000,60.921461


In [13]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) < 20)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Buy"]= np.where((self.df.trigger) & (self.df.K < 0.20), 1, 0)


In [14]:
inst = Signals(df, 2)

In [15]:
inst.decide()

In [16]:
df.tail(151)

,Open,High,Low,Close,Volume,K,RSI,trigger,Buy
Time,,,,,,,,,
2022-11-30 01:45:00,16837.19,16893.15,16824.70,16875.85,73.422423,0.799960,77.884861,0,0
2022-11-30 02:00:00,16875.85,16974.44,16875.82,16895.90,123.635947,0.821507,78.731701,0,0
2022-11-30 02:15:00,16895.90,17080.00,16895.90,17020.42,304.773651,0.931843,83.068081,0,0
2022-11-30 02:30:00,17020.42,17034.17,16986.35,17000.30,116.574774,0.859424,80.221909,0,0
2022-11-30 02:45:00,17000.30,17019.08,16958.32,16959.03,97.098241,0.715804,74.577410,0,0
...,...,...,...,...,...,...,...,...,...
2022-12-01 14:15:00,17084.19,17084.19,17025.80,17031.79,123.810470,0.000000,33.178663,0,0
2022-12-01 14:30:00,17031.79,17087.12,17026.98,17070.34,90.611994,0.389421,43.362426,0,0
2022-12-01 14:45:00,17070.34,17142.84,17070.34,17107.77,68.125404,0.687117,51.147459,0,0


In [17]:
df[df.Buy==1]

,Open,High,Low,Close,Volume,K,RSI,trigger,Buy
Time,,,,,,,,,
2022-11-21 19:30:00,15814.55,15860.00,15593.48,15676.99,417.496872,0.000000,19.240914,1,1
2022-11-28 01:15:00,16388.00,16396.00,16192.95,16234.51,372.109590,0.000000,16.341904,1,1
2022-11-28 01:30:00,16234.51,16240.00,16114.32,16154.54,274.664868,0.000000,13.496908,1,1
2022-11-28 01:45:00,16154.54,16167.00,16099.47,16147.83,106.145603,0.000000,13.287875,1,1
2022-11-28 02:00:00,16147.83,16152.53,16056.02,16140.34,149.339512,0.000000,13.045008,1,1
2022-11-28 02:15:00,16140.34,16192.80,16103.35,16133.17,69.961093,0.000000,12.803755,1,1
2022-11-28 02:30:00,16133.17,16185.45,16119.47,16143.32,73.710590,0.090769,15.194770,1,1
2022-11-28 02:45:00,16143.32,16152.94,16106.03,16119.90,56.347533,0.053972,14.225473,1,1


In [19]:
def strategy(qty, open_position = False):
    df= get15minutedata()
    apply_technicals(df)
    inst = Signals(df, 5)
    inst.decide()
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1]:
        order = session_auth.place_active_order(symbol="BTCUSDT",
                                                side="Buy",
                                                type="MARKET",
                                                qty= qty,
                                                timeInForce="GTC")
        print(order)
        buyprice = float(order['result']['price'])
        open_position = True
    while open_position:
        time.sleep(2)
        df = get15minutedata()
        print(f'Current Close ' + str(df.Close.iloc[-1]))
        print(f'Potential Target ' + str(buyprice * 1.02))
        #print(f'Current Target 2 ' + str(df.RSI[-1] > 45))
        print(f'Current Stop ' + str(buyprice * 0.98))
        if df.Close[-1] <= buyprice* 0.98 or df.Close[-1] >= 1.02 * buyprice or df.RSI[-1] > 45:
            order = session_auth.place_active_order(symbol="BTCUSDT",
                                                side="Sell",
                                                type="MARKET",
                                                qty= qty,
                                                timeInForce="GTC")
            print(order)
            break

In [20]:
strategy(0.001)

Current Close is 16994.24


In [21]:
while True: 
    strategy(0.001)
    time.sleep(1)

Current Close is 16994.24
Current Close is 16986.49
Current Close is 16986.49
Current Close is 16986.49
Current Close is 16986.49
Current Close is 16982.59
Current Close is 16982.59
Current Close is 16982.59
Current Close is 16982.59
Current Close is 16960.01
Current Close is 16960.01
Current Close is 16960.01
Current Close is 16970.0
Current Close is 16970.0
Current Close is 16970.0
Current Close is 16970.0
Current Close is 16969.27
Current Close is 16969.27
Current Close is 16969.27
Current Close is 16969.27
Current Close is 16969.27
Current Close is 16969.31
Current Close is 16969.31
Current Close is 16969.31
Current Close is 16969.31
Current Close is 16969.02
Current Close is 16969.02
Current Close is 16969.02
Current Close is 16970.28
Current Close is 16970.28
Current Close is 16970.28
Current Close is 16970.28
Current Close is 16970.28
Current Close is 16970.34
Current Close is 16970.34
Current Close is 16970.34
Current Close is 16974.22
Current Close is 16974.22
Current Close is

KeyboardInterrupt: 

In [ ]:
#Get active order 
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
print(session_auth.get_active_order(
    orderId=""
))

In [ ]:
#Cancel active order
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
print(session_auth.cancel_active_order(
    orderId=""
))